<a href="https://www.kaggle.com/code/albertobircoci/world-s-tallest-persons-data-cleaning-analysis?scriptVersionId=185415207" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

I concatenated the datasets for the analysis.

In [ ]:
import numpy as np 
import pandas as pd

path_men = '/kaggle/input/worlds-tallest-men-and-women/tallest_men.csv'
path_women = '/kaggle/input/worlds-tallest-men-and-women/tallest_women.csv'

men_df = pd.read_csv(path_men, sep=';')
women_df = pd.read_csv(path_women, sep=';')
men_df['Gender'] = 0
women_df['Gender'] = 1  
combined_df = pd.concat([men_df, women_df], ignore_index=True)

print(combined_df.head())


I dropped the 'Imperial' column from the combined dataframe 

In [ ]:
combined_df = combined_df.drop(columns=['Imperial'])
print(combined_df.head())

I applied a function to determine the 'isAlive' status based on patterns in the 'Lifespan' column, adding the result as a new column in combined_df. Then, I printed rows where the 'isAlive' status couldn't be classified.

In [ ]:
import re

def determine_is_alive(lifespan):
    if pd.isna(lifespan):
        return None
    if re.search(r'born|c. born', lifespan, re.IGNORECASE):
        return 1
    elif re.search(r'\d{4}–\d{4}', lifespan):
        return 0
    else:
        return None
    
combined_df['isAlive'] = combined_df['Lifespan'].apply(determine_is_alive)
unclassified = combined_df[combined_df['isAlive'].isna()]

print("Unclassified rows:")
print(unclassified[['Name', 'Lifespan']])

I removed rows with missing values in the 'Name' column from combined_df, filled missing values in the 'isAlive' column with 0, and printed the first few rows of the updated dataframe. I then identified and printed rows where the 'isAlive' status could not be classified.

In [ ]:
combined_df = combined_df.dropna(subset=['Name'])

combined_df['isAlive'] = combined_df['isAlive'].fillna(0)

print(combined_df.head())

unclassified = combined_df[combined_df['isAlive'].isna()]
print("Unclassified rows after processing:")
print(unclassified[['Name', 'Lifespan']])


Data Extraction and Calculation:

Two functions, extract_year_born and extract_year_died, extract birth and death years from the 'Lifespan' column.
The calculate_age function computes age based on extracted birth and death years.
These functions are applied to create new columns 'YearBorn', 'YearDied', and 'Age' in combined_df.
Handling Special Cases:

The handle_special_cases function addresses specific patterns in the 'Lifespan' column that couldn't be handled by regular extraction methods.
Special cases include patterns like 'born year' and certain date ranges.
This function fills in missing values in the 'Age' column where applicable.
Identification and Reporting:

Rows with missing values in any of 'YearBorn', 'YearDied', or 'Age' columns are identified as unclassified.
Rows containing special cases in the 'Lifespan' column are identified separately for manual inspection.
Results are printed to show the first few rows of combined_df, unclassified rows after column additions, and special cases for manual inspection.

In [ ]:
def extract_year_born(lifespan):
    if pd.isna(lifespan):
        return None
    match = re.match(r'(\d{4})', lifespan)
    if match:
        return int(match.group(1))
    return None

def extract_year_died(lifespan):
    if pd.isna(lifespan):
        return None
    match = re.search(r'–(\d{4})', lifespan)
    if match:
        return int(match.group(1))
    return None

def calculate_age(year_born, year_died):
    if pd.isna(year_born):
        return None
    if pd.isna(year_died):
        return None
    return year_died - year_born

combined_df['YearBorn'] = combined_df['Lifespan'].apply(extract_year_born)
combined_df['YearDied'] = combined_df['Lifespan'].apply(extract_year_died)
combined_df['Age'] = combined_df.apply(lambda row: calculate_age(row['YearBorn'], row['YearDied']), axis=1)

def handle_special_cases(lifespan):
    if pd.isna(lifespan):
        return None
    if re.search(r'born \d{4}', lifespan):
        match = re.search(r'born (\d{4})', lifespan)
        if match:
            return 2024 - int(match.group(1))
    if re.search(r'\d{4}–\d{4}.*\d{4}', lifespan):
        match = re.search(r'(\d{4})–(\d{4})', lifespan)
        if match:
            return int(match.group(2)) - int(match.group(1))
    return None

combined_df.loc[combined_df['Age'].isna(), 'Age'] = combined_df.loc[combined_df['Age'].isna(), 'Lifespan'].apply(handle_special_cases)

def is_special_case(lifespan):
    if pd.isna(lifespan):
        return False
    if re.search(r'\d{4}–\d{4}', lifespan):
        return False
    if re.search(r'born \d{4}', lifespan):
        return False
    if re.search(r'\d{4}–\?', lifespan):
        return True
    if re.search(r'fl\. \?\–\d{4}', lifespan):
        return True
    if re.search(r'\d{4}/\d{2} – \d{4}/\d{2}', lifespan):
        return True
    return False

unclassified = combined_df[combined_df[['YearBorn', 'YearDied', 'Age']].isna().any(axis=1)]
special_cases = combined_df[combined_df['Lifespan'].apply(is_special_case)]
print(combined_df.head())
print("Unclassified rows after adding new columns:")
print(unclassified[['Name', 'Lifespan', 'YearBorn', 'YearDied', 'Age']])
print("Special cases for manual inspection:")
print(special_cases[['Name', 'Lifespan', 'YearBorn', 'YearDied', 'Age']])


I updated the 'Age' column for entries without a death year by calculating their age based on their birth year and the current year. Additionally, I identified and printed rows where the 'Lifespan' column contains the phrase 'born 1990s' for further review.

In [ ]:
from datetime import datetime

today = datetime(2024, 6, 25)
combined_df.loc[combined_df['YearDied'].isna() & combined_df['YearBorn'].notna(), 'Age'] = today.year - combined_df['YearBorn']
unclassified = combined_df[combined_df['Lifespan'].str.contains('born 1990s', na=False)]
print(combined_df.head())
print("Unclassified rows after adding new columns:")
print(unclassified[['Name', 'Lifespan', 'YearBorn', 'YearDied', 'Age']])

In [ ]:
today = datetime(2024, 6, 25)
combined_df.loc[combined_df['YearDied'].isna() & combined_df['YearBorn'].notna(), 'Age'] = today.year - combined_df['YearBorn']
unclassified = combined_df[combined_df['Lifespan'].str.contains(r'born \d{4}s', na=False)]
special_cases = combined_df[combined_df['Lifespan'].apply(is_special_case)]

print("Unclassified rows after adding new columns:")
print(unclassified[['Name', 'Lifespan', 'YearBorn', 'YearDied', 'Age']])

print("Special cases for manual inspection:")
print(special_cases[['Name', 'Lifespan', 'YearBorn', 'YearDied', 'Age']])

In [ ]:
combined_df.loc[unclassified.index, 'Age'] = None
combined_df.loc[special_cases.index, 'Age'] = None

In [ ]:
# selected_columns = ['Lifespan', 'Gender', 'isAlive', 'YearBorn', 'YearDied', 'Age']
# selected_df = combined_df[selected_columns]

# print(selected_df.to_string(index=False))


In [ ]:
def handle_age_correction(row):
    lifespan = row['Lifespan']
    age = row['Age']    
    if pd.isna(lifespan):
        return age    
    if re.search(r'\((\d{1,3})\)', lifespan):
        match_lifespan = re.search(r'\((\d{1,3})\)', lifespan)
        age_in_parentheses = int(match_lifespan.group(1))        
        if age != age_in_parentheses:
            return age_in_parentheses  
    return age

combined_df['Age'] = combined_df.apply(handle_age_correction, axis=1)
selected_columns = ['Lifespan', 'Gender', 'isAlive', 'YearBorn', 'YearDied', 'Age']
selected_df = combined_df[selected_columns]
#print(selected_df.to_string(index=False))


The code extracts and corrects the birth year ('YearBorn') from the 'Lifespan' column in combined_df. It applies functions to handle various patterns, including missing values and specific formats like 'born YYYY' and 'born YYYYs', updating the dataframe accordingly. The selected columns ('Lifespan', 'Gender', 'isAlive', 'YearBorn', 'YearDied', 'Age') are then printed for analysis or further processing.

In [ ]:
def extract_year_born(lifespan):
    if pd.isna(lifespan):
        return None
    match = re.search(r'born (\d{4})', lifespan)
    if match:
        return int(match.group(1))
    return None

def handle_year_born_correction(row):
    lifespan = row['Lifespan']
    year_born = row['YearBorn']  
    if pd.isna(lifespan):
        return year_born   
    if re.search(r'born \d{4}', lifespan):
        match = re.search(r'born (\d{4})', lifespan)
        year_born_from_lifespan = int(match.group(1))      
        if re.search(r'born \d{4}s', lifespan):
            return year_born        
        return year_born_from_lifespan 
    return year_born

combined_df['YearBorn'] = combined_df.apply(handle_year_born_correction, axis=1)
selected_columns = ['Lifespan', 'Gender', 'isAlive', 'YearBorn', 'YearDied', 'Age']
selected_df = combined_df[selected_columns]
#print(selected_df.to_string(index=False))



In [ ]:
columns_to_drop = ['Note', 'Lifespan']
combined_df = combined_df.drop(columns=columns_to_drop, axis=1)

print(combined_df.head())

Choropleth map using GeoPandas and Matplotlib to visualize the distribution of total people per country.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

corrections = {'United States': 'United States of America'}
combined_df['Country'] = combined_df['Country'].replace(corrections)

total_people_per_country = combined_df['Country'].value_counts().reset_index()
total_people_per_country.columns = ['Country', 'TotalPeople']
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.merge(total_people_per_country, how='left', left_on='name', right_on='Country')
world['TotalPeople'] = world['TotalPeople'].fillna(0) 

colors = [(1, 1, 0), (1, 0, 0)]
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=256)
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.plot(column='TotalPeople', ax=ax, legend=True, cmap=cmap, legend_kwds={'label': "Total People", 'orientation': "horizontal"})
plt.title('World Map Colored by Total People')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


This code uses Seaborn and Matplotlib to create a histogram plot showing the distribution of heights in centimeters (Height_cm) from combined_df. It cleans the 'Metric' column to extract numerical values, converts them to float, and then plots a histogram with 10 bins and a kernel density estimate (KDE) overlay. The plot provides a visual representation of the distribution of heights.

In [ ]:
import seaborn as sns

combined_df['Height_cm'] = combined_df['Metric'].str.replace('\s+cm', '', regex=True).astype(float)
plt.figure(figsize=(20, 6))
sns.histplot(data=combined_df['Height_cm'], bins=10, kde=True, color='blue')
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Height')
plt.show()


This code creates side-by-side histogram plots using Seaborn and Matplotlib to compare the distribution of heights in centimeters (Height_cm) between males and females from combined_df. It filters the dataframe into male_df and female_df, then plots separate histograms with 10 bins and KDE overlays for each gender. The plots provide visual comparisons of height distributions between males and females.

In [ ]:
male_df = combined_df[combined_df['Gender'] == 0]
female_df = combined_df[combined_df['Gender'] == 1]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
sns.histplot(data=male_df['Height_cm'], bins=10, kde=True, color='blue', ax=ax1)
ax1.set_title('Distribution of Height for Males')
ax1.set_ylabel('Frequency')

sns.histplot(data=female_df['Height_cm'], bins=10, kde=True, color='pink', ax=ax2)
ax2.set_title('Distribution of Height for Females')
ax2.set_xlabel('Height (cm)')
ax2.set_ylabel('Frequency')
plt.tight_layout()
plt.show()


Histogram for alive persons from the datasets. 

In [ ]:
male_df = combined_df[(combined_df['Gender'] == 0) & (combined_df['isAlive'] == 1)]
female_df = combined_df[(combined_df['Gender'] == 1) & (combined_df['isAlive'] == 1)]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
sns.histplot(data=male_df['Height_cm'], bins=10, kde=True, color='blue', ax=ax1)
ax1.set_title('Distribution of Height for Males')
ax1.set_ylabel('Frequency')

sns.histplot(data=female_df['Height_cm'], bins=10, kde=True, color='pink', ax=ax2)
ax2.set_title('Distribution of Height for Females')
ax2.set_xlabel('Height (cm)')
ax2.set_ylabel('Frequency')
plt.tight_layout()
plt.show()


In [ ]:
male_df = combined_df[(combined_df['Gender'] == 0) & (combined_df['isAlive'] == 0)]
female_df = combined_df[(combined_df['Gender'] == 1) & (combined_df['isAlive'] == 0)]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
sns.histplot(data=male_df['Height_cm'], bins=10, kde=True, color='blue', ax=ax1)
ax1.set_title('Distribution of Height for Males')
ax1.set_ylabel('Frequency')

sns.histplot(data=female_df['Height_cm'], bins=10, kde=True, color='pink', ax=ax2)
ax2.set_title('Distribution of Height for Females')
ax2.set_xlabel('Height (cm)')
ax2.set_ylabel('Frequency')
plt.tight_layout()
plt.show()


A scatter plot with a regression line, illustrating the relationship between age (Age) and height in centimeters (Height_cm) from combined_df. 
The plot includes scatter points in blue with reduced opacity (alpha=0.7), a red dashed regression line, and axis labels indicating age and height. The visualization aims to show any potential trend or correlation between age and height.

In [ ]:
combined_df.loc[24, 'Age'] = np.nan 

plt.figure(figsize=(10, 6))
sns.regplot(x='Age', y='Height_cm', data=combined_df, scatter_kws={'color': 'b', 'alpha': 0.7}, line_kws={'color': 'r', 'linestyle': '--'})
plt.title('Age vs. Height with Trend Line')
plt.xlabel('Age (years)')
plt.ylabel('Height (cm)')
plt.grid(True)
plt.tight_layout()

plt.show()